In [66]:
# 3. Import libraries and modules
import numpy as np
np.random.seed(123)  # for reproducibility
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import model_from_json

pieceMapW = {
  '1': 0,
  'p': 1,
  'r': 2,
  'n': 3,
  'b': 4,
  'q': 5,
  'k': 6,
  'P': 7,
  'R': 8,
  'N': 9,
  'B': 10,
  'Q': 11,
  'K': 12
}

pieceMapB = {
  '1': 0,
  'P': 1,
  'R': 2,
  'N': 3,
  'B': 4,
  'Q': 5,
  'K': 6,
  'p': 7,
  'r': 8,
  'n': 9,
  'b': 10,
  'q': 11,
  'k': 12  
}



def loadData(dataset,nRows):
    X_vals = []
    Y_vals = []
    i = 0
    pm = pieceMapW
    with open(dataset,'r') as data:
        boardLine = data.readline()

        while boardLine and i < nRows:
            boardData = boardLine.split(':')
            boardR = boardData[0].split('/')
            board = boardR[0:8]
            board[7] = board[7][0:8]
            who = boardR[7][9]
            
            if who == 'w':
                nBoard = []
                nLine = []
                for line in board:
                    for piece in line:
                        nLine.append(pm[piece])
                    nBoard.append(nLine)
                    nLine = []

                X_vals.append(nBoard)
                #Y_vals.append(1 if float(boardData[2]) > 0 else 0)
                pc = float(boardData[1])
                Y_vals.append(pc)
#                 print(boardLine)
#                 print(nBoard)
#                 print(pc,"\n")
            boardLine = data.readline()
            i += 1
        X_vals = np.array(X_vals)
        Y_vals = np.array(Y_vals)
#         print('wat',X_vals.shape)
#         print('Y', Y_vals[4:10])
        #Xraw = data.split('\n').map(lambda r: r.split('/'))

    # 4. Load pre-shuffled MNIST data into train and test sets

#     split = nRows // 2
#     (X_train, y_train) = X_vals[0:split], Y_vals[0: split].reshape(split,1)
#     (X_test, y_test) = X_vals[split :], Y_vals[split :].reshape(split,1)
    #(X_train, y_train), (X_test, y_test) = mnist.load_data()


    #n = 6000
    #X_train = X_train[0:n]
    #y_train = y_train[0:n]
    #X_test = X_test[0:n]
    #y_test = y_test[0:n]

    # 5. Preprocess input data
    X_vals = X_vals.reshape(X_vals.shape[0], *chess_shape)
    
#     X_train = X_train.reshape(X_train.shape[0], *chess_shape)
#     X_test = X_test.reshape(X_test.shape[0], *chess_shape)
    
    #X_train = X_train.astype('float32')
    #X_test = X_test.astype('float32')
    #X_train /= 12
    #X_test /= 12


    print("done loading: ", dataset)
    return X_vals, Y_vals

chess_shape = (1, 8, 8)


# 7. Define model architecture
model = Sequential()
model.add(Dense(, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# model.add(Convolution2D(64, (8, 8), activation='tanh', input_shape=chess_shape, data_format='channels_first'))
# model.add(Convolution2D(64, (3, 3), activation='relu', input_shape=chess_shape, data_format='channels_first'))
# model.add(Dropout(0.25))
# model.add(Convolution2D(96, (3, 3), activation='relu'))
# model.add(Dropout(0.25))
# model.add(Convolution2D(256, (3, 3), activation='relu'))
# model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(1, activation='softmax'))

print('compile')
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
print('summary')
model.summary()



# X_train, Y_train = loadData('data/smallSample2.data',2)
# model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)
# score = model.evaluate(X_train[0:2], Y_train[0:2, verbose=0)
# print("score: ",score)
# print(X_train[0:2])
# print("orginal data: ",Y_train[0:2])
# print(model.predict(X_train[0:2]))

# #Train
print("loading blitzz")
X_train, Y_train = loadData('data/blitzz.data',13000)
# # X_train, Y_train = loadData('data/blitzz.data',1300000)
model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

print("loading standar")
X_test, Y_test = loadData('data/standar.data',64000)
# X_test, Y_test = loadData('data/standar.data',640000)
print("evaluating")
score = model.evaluate(X_test, Y_test, verbose=0)
print("score: ",score)
# print(Y_test[0:10])
# print(model.predict(Y_test[0:10]))
# print("loading lightning")
# X_train, Y_train = loadData('data/lightning.data',890000)
# model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

# print("evaluating")
# score = model.evaluate(X_test, Y_test, verbose=0)
# print("score: ",score)

# print("loading titled")
# X_train, Y_train = loadData('data/titled.data',3200000)
# # 10. Evaluate model on test data

# print("evaluating")
# score = model.evaluate(X_test, Y_test, verbose=0)
# print("score: ",score)

print("Done\n")

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")

compile
summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_187 (Conv2D)          (None, 64, 1, 1)          4160      
_________________________________________________________________
flatten_63 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 65        
Total params: 4,225
Trainable params: 4,225
Non-trainable params: 0
_________________________________________________________________
loading blitzz
done loading:  data/blitzz.data
Epoch 1/10
6475/6475 [==============================] - 2s 290us/step - loss: 161.3419 - acc: 0.0015
Epoch 2/10
6475/6475 [==============================] - 1s 83us/step - loss: 161.3419 - acc: 0.0015
Epoch 3/10
6475/6475 [==============================] - 0s 72us/step - loss: 161.3419 - acc: 0.0015
Epoch 4/10
6475/6475 [==================

In [43]:
print(Y_train)
model.predict(X_train)

[ 1.99  1.99  1.99  1.99  1.99  1.99  1.99  1.99  1.99  1.99  1.99  1.99
  1.99  1.99  1.99  1.99  1.99  1.99  1.99  1.99]


array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32)

25000/25000 [==============================] - 1s 38us/step
score is [0.8661627592849731, 0.24479999999999999]
